In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [3]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [4]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization

# Main

In [5]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
#         {"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
        {"instance": category, "method": ['cat_customer_idx','customer_idx_merge_enterprise','historical_cnt_mean'], "params": False},
    ]
}

## preprocessing

In [6]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

## encoding

In [8]:
encode = Encode()

In [9]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [10]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [11]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [12]:
# label encoding
encode.set_up() # 전역 변수 초기화
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [13]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','historial_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [14]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## train

In [140]:
import math
from six.moves import xrange

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                #( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (1-2*p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 3*(p)*(targets[index] - p)
                der2 = 3*(1-2*p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 3배 페널티
                der1 = 3*(1-p)*(targets[index] - p)
                der2 = 3*(2*p-1)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [145]:
model = _model.catboost
model.set_params(iterations=5000,
                depth=12,
                thread_count=3,
                learning_rate=1e-3,
                loss_function=LoglossObjective(),
                eval_metric='PRAUC',
                early_stopping_rounds=400,
                verbose=1000)

In [75]:
# dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
# model = _model.decision_tree
# model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

In [40]:
model = _model.xgboost
# model.set_params(**{'n_estimators': 2393,
#  'max_depth': 16,
#  'min_child_weight': 2,
#  'learning_rate': 0.06783336462585174,
#  'subsample': 0.9511145375386896,
#  'gamma': 0.16846211770550093,
#  'colsample_bytree': 0.6662896237544925,
#  'colsample_bylevel': 0.8781042717330898,
#  'colsample_bynode': 0.741271661588135,
# 'random_state':42})
model.set_params(random_state=42)
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

In [18]:
# model = _model.lightgbm
# model.set_params(random_state = 42)

LGBMClassifier(n_estimators=150, random_state=42)

In [175]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [147]:
# Y = np.where(Y == False, 0, 1) # catboost

In [20]:
# oversampling
from imblearn.over_sampling import SMOTE, ADASYN
adasyn = SMOTE(sampling_strategy=0.2,random_state=42)
X, Y = adasyn.fit_resample(X, Y)

In [148]:
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
train_result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
models = [] # 모델을 저장할 리스트

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련

    clf = model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)

    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)

    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)

print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

0it [00:00, ?it/s]

0:	learn: 0.7686404	total: 408ms	remaining: 33m 59s
1000:	learn: 0.9213581	total: 49.9s	remaining: 3m 19s
2000:	learn: 0.9411811	total: 1m 37s	remaining: 2m 25s
3000:	learn: 0.9525657	total: 2m 24s	remaining: 1m 36s
4000:	learn: 0.9602446	total: 3m 11s	remaining: 47.8s
4999:	learn: 0.9660071	total: 3m 57s	remaining: 0us


1it [04:00, 240.65s/it]

0:	learn: 0.6856965	total: 304ms	remaining: 25m 18s
1000:	learn: 0.9285252	total: 46.7s	remaining: 3m 6s
2000:	learn: 0.9452524	total: 1m 33s	remaining: 2m 19s
3000:	learn: 0.9551510	total: 2m 19s	remaining: 1m 33s
4000:	learn: 0.9624888	total: 3m 6s	remaining: 46.6s
4999:	learn: 0.9680265	total: 3m 53s	remaining: 0us


2it [07:57, 238.16s/it]

...

0:	learn: 0.7725322	total: 291ms	remaining: 24m 15s
1000:	learn: 0.9271261	total: 46.2s	remaining: 3m 4s
2000:	learn: 0.9435955	total: 1m 32s	remaining: 2m 18s
3000:	learn: 0.9537073	total: 2m 20s	remaining: 1m 33s
4000:	learn: 0.9607658	total: 3m 8s	remaining: 46.9s
4999:	learn: 0.9664361	total: 3m 54s	remaining: 0us


4it [15:51, 237.60s/it]

0:	learn: 0.7694271	total: 276ms	remaining: 23m 1s
1000:	learn: 0.9193848	total: 47.9s	remaining: 3m 11s
2000:	learn: 0.9393181	total: 1m 35s	remaining: 2m 23s
3000:	learn: 0.9512335	total: 2m 23s	remaining: 1m 35s
4000:	learn: 0.9592304	total: 3m 10s	remaining: 47.5s
4999:	learn: 0.9654067	total: 3m 57s	remaining: 0us


5it [19:52, 238.53s/it]

----[K-Fold Train Score]-----
f1 score : 0.9173 / STD: (+/- 0.0008)
precision score : 0.9007 / STD: (+/- 0.0034)
recall score : 0.9345 / STD: (+/- 0.0025)
roc_auc_score score : 0.9628 / STD: (+/- 0.0011)
----[K-Fold Validation Score]-----
f1 score : 0.8177 / STD: (+/- 0.0049)
precision score : 0.8145 / STD: (+/- 0.0103)
recall score : 0.8210 / STD: (+/- 0.0088)
roc_auc_score score : 0.9022 / STD: (+/- 0.0042)


In [177]:
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_desc_length: 32.84827170456088
customer_interest: 27.302141973886435
lead_owner_converted_ratio: 22.52998952549251
new_customer_idx: 14.402168028616286
continent: 2.8391408893840366
new_customer_type2: 0.011096853797201472
country: 0.009991666151968073
new_customer_type1: 0.008545707948952626
business_subarea: 0.008136737820490186
ver_win_rate_x: 0.006096719090939381
business_area: 0.005014102563013591
job_function: 0.003561107429170348
category_3: 0.0035173840857693306
seniority_level: 0.0034062472612920915
ver_win_ratio_per_bu: 0.0028763242142571415
com_reg_ver_win_rate: 0.002246081869386564
historical_existing_cnt: 0.002200189728662745
category_1: 0.002154894432089011
category_2: 0.002109207991744617
bant_submit: 0.002057300203571304
less_timeline: 0.001701753448327613
new_expected_timeline: 0.0016774574954834833
business_unit: 0.001570263435891192
new_inquiry_type: 0.0002441203604682599
enterprise: 8.375873116804892e-05


## test & submission

In [178]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [179]:
# predict
pred = model.predict(test_drop)
pred = np.where(pred == 0, False, True)
sum(pred)

In [174]:
# threshold !!
y_pred_proba = model.predict_proba(test_drop)
empty = np.empty(y_pred_proba.shape[0], dtype=bool) # 새 빈 배열 만들기
threshold = 0.60
for idx, val in enumerate(y_pred_proba):
    if np.max(val) < threshold: # 배열의 요소의 max가 0.9보다 작으면 무조건 True
        empty[idx] = True
    else:
        if np.argmax(val): # idx:1(True) 이면 True
            empty[idx] = True
        else: # idx:0(False) 이면 False
            empty[idx] = False
print(sum(empty))
pred = empty
# Y_pred = y_pred_proba[:,0] # False 값만뽑기
# sum(np.where(Y_pred > threshold, False, True))

1314


In [168]:
pred3 = pred  #pred1 718 | pred2 1142 | pred3 820

In [183]:
sum(pred1)

1469

In [185]:
su = 0
for i,j in zip(pred3, pred1):
    if i != j:
        su +=1
su

249

In [ ]:
# pred1_ = pred1.reshape(-1,1)
# pred2_ = pred2.reshape(-1,1)
# pred3_ = pred3.reshape(-1,1)
# combine = np.hstack((pred1_, pred2_, pred3_))
# combine_ = np.round(combine.mean(axis=1))

In [170]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [171]:
sum(pred)

1314